In [ ]:
pip install neo4j

In [2]:
import os

from langchain.chat_models import ChatOpenAI
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph

os.environ['OPENAI_API_KEY'] = ""

# Neo4j 연결
graph = Neo4jGraph(
    url="", 
    username="", 
    password=""
)

chain = GraphCypherQAChain.from_llm(
ChatOpenAI(temperature=0), graph=graph, verbose=True,
)

## 1. 정상시나리오

#### 1) 예정된 waypoint, vertipad, vertiport 스케줄 확인

In [4]:
chain.run("""
What time and which waypoint the UAM1 is on between 1pm and 2pm? Tell me all of them.
""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (uam:UAM {uam_id: "UAM1"})-[:ON_WAYPOINT]->(waypoint:Waypoint)-[:ON_TIME]->(time:Time)
WHERE time.time >= time("13:00:00") AND time.time <= time("14:00:00")
RETURN time.time, waypoint.waypoint_id
Full Context:
[{'time.time': neo4j.time.Time(13, 0, 0, 0, tzinfo=<UTC>), 'waypoint.waypoint_id': '고_김_1'}, {'time.time': neo4j.time.Time(13, 5, 0, 0, tzinfo=<UTC>), 'waypoint.waypoint_id': '고_김_2'}, {'time.time': neo4j.time.Time(13, 20, 0, 0, tzinfo=<UTC>), 'waypoint.waypoint_id': '김_고_1'}, {'time.time': neo4j.time.Time(13, 25, 0, 0, tzinfo=<UTC>), 'waypoint.waypoint_id': '김_고_2'}, {'time.time': neo4j.time.Time(13, 50, 0, 0, tzinfo=<UTC>), 'waypoint.waypoint_id': '고_김_1'}, {'time.time': neo4j.time.Time(13, 55, 0, 0, tzinfo=<UTC>), 'waypoint.waypoint_id': '고_김_2'}]

> Finished chain.


'Between 1pm and 2pm, the UAM1 was on the following waypoints at the specified times:\n- At 1:00pm, it was at waypoint 고_김_1.\n- At 1:05pm, it was at waypoint 고_김_2.\n- At 1:20pm, it was at waypoint 김_고_1.\n- At 1:25pm, it was at waypoint 김_고_2.\n- At 1:50pm, it was at waypoint 고_김_1.\n- At 1:55pm, it was at waypoint 고_김_2.'

In [6]:
chain.run("""
What time the UAM1 is on vertipad '김포공항_1' after 1pm?
""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (uam:UAM {uam_id: 'UAM1'})-[:ON_VERTIPAD]->(vertipad:Vertipad {vertipad_id: '김포공항_1'})-[:ON_TIME]->(time:Time)
WHERE time.time > time('13:00:00')
RETURN time.time
Full Context:
[{'time.time': neo4j.time.Time(13, 10, 0, 0, tzinfo=<UTC>)}, {'time.time': neo4j.time.Time(14, 0, 0, 0, tzinfo=<UTC>)}, {'time.time': neo4j.time.Time(14, 50, 0, 0, tzinfo=<UTC>)}, {'time.time': neo4j.time.Time(15, 40, 0, 0, tzinfo=<UTC>)}, {'time.time': neo4j.time.Time(16, 30, 0, 0, tzinfo=<UTC>)}, {'time.time': neo4j.time.Time(17, 20, 0, 0, tzinfo=<UTC>)}, {'time.time': neo4j.time.Time(18, 10, 0, 0, tzinfo=<UTC>)}]

> Finished chain.


"The UAM1 is on vertipad '김포공항_1' at the following times after 1pm: 2pm, 2:50pm, 3:40pm, 4:30pm, 5:20pm, and 6:10pm."

In [54]:
chain.run("""
What time the UAM1 is in vertiport '김포공항' after 1pm?
""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (uam:UAM {uam_id: 'UAM1'})-[:ON_VERTIPAD]->(vertipad:Vertipad)-[:IN_VERTIPORT]->(vertiport:Vertiport {vertiport_id: '김포공항'})-[:ON_TIME]->(time:Time)
WHERE time.time > time('13:00')
RETURN time.time
Full Context:
[{'time.time': neo4j.time.Time(13, 10, 0, 0, tzinfo=<UTC>)}, {'time.time': neo4j.time.Time(13, 10, 0, 0, tzinfo=<UTC>)}, {'time.time': neo4j.time.Time(14, 0, 0, 0, tzinfo=<UTC>)}, {'time.time': neo4j.time.Time(14, 0, 0, 0, tzinfo=<UTC>)}, {'time.time': neo4j.time.Time(14, 50, 0, 0, tzinfo=<UTC>)}, {'time.time': neo4j.time.Time(14, 50, 0, 0, tzinfo=<UTC>)}, {'time.time': neo4j.time.Time(15, 40, 0, 0, tzinfo=<UTC>)}, {'time.time': neo4j.time.Time(15, 40, 0, 0, tzinfo=<UTC>)}, {'time.time': neo4j.time.Time(16, 30, 0, 0, tzinfo=<UTC>)}, {'time.time': neo4j.time.Time(16, 30, 0, 0, tzinfo=<UTC>)}]

> Finished chain.


"The UAM1 is in vertiport '김포공항' at 1:10pm, 2:00pm, 2:50pm, 3:40pm, and 4:30pm."

#### 2) 예정된 시간에 UAM이 도착할 vertiport가 위치한 영역이 비행 가능한 환경인지 확인

In [31]:
chain.run("""
At 1:10pm, is the area that vertiport '김포공항' is located in available?
""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (v:Vertiport {vertiport_id: '김포공항'})-[:IN_AREA]->(a:Area)-[:ON_TIME]->(t:Time {time: time('13:10')})
RETURN a.availability
Full Context:
[{'a.availability': True}, {'a.availability': True}]

> Finished chain.


"Yes, at 1:10pm, the area that vertiport '김포공항' is located in is available."

## 2. 비정상 시나리오

### 2-1. 스케줄 상 시간을 준수하지 못하는 경우

    1. waypoint 5-1을 10분 지연되어 통과했으므로 비정상 운행으로 판단
    2. 해당 스케줄에서 도착지인 여의도 vertiport에서의 time, vertipad를 확인
    3. time1에 지연된 시간 10분을 더한 시점을 time2라 하고 time2 시점에서 여의도 vertiport와 vertipad1의 상태를 확인, vertiport를 포함한 area는 available 가정

        3-1) vertipad1을 uam1이 차지하고 있거나 비어있을 것으로 예정되어 있는 경우
            
            → result = 예상도착시간 : time2, 착륙가능여부 : true, 착륙위치 : vertipad1
            
        3-2) vertipad1이 uam1이 아닌 다른 기체가 차지하고 있을 것으로 예정되어 있는 경우
            
            → 여의도 vertiport의 비어있는 vertipad가 있는지 확인 → vertipad2
            
            → result = 예상도착시간 : time2, 착륙가능여부 : true, 착륙위치 : vertipad2

#### 1) 예정된 스케줄 확인

In [61]:
chain.run("""
What time and which vertipads the UAM1 on after 1pm?
""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (uam:UAM {uam_id: "UAM1"})-[:ON_VERTIPAD]->(vertipad:Vertipad)-[:ON_TIME]->(time:Time)
WHERE time.time > time("13:00")
RETURN time, vertipad.vertipad_id
Full Context:
[{'time': {'time': neo4j.time.Time(13, 10, 0, 0, tzinfo=<UTC>)}, 'vertipad.vertipad_id': '김포공항_1'}, {'time': {'time': neo4j.time.Time(13, 35, 0, 0, tzinfo=<UTC>)}, 'vertipad.vertipad_id': '고양킨텍스_1'}, {'time': {'time': neo4j.time.Time(14, 0, 0, 0, tzinfo=<UTC>)}, 'vertipad.vertipad_id': '김포공항_1'}, {'time': {'time': neo4j.time.Time(14, 25, 0, 0, tzinfo=<UTC>)}, 'vertipad.vertipad_id': '고양킨텍스_1'}, {'time': {'time': neo4j.time.Time(14, 50, 0, 0, tzinfo=<UTC>)}, 'vertipad.vertipad_id': '김포공항_1'}, {'time': {'time': neo4j.time.Time(15, 15, 0, 0, tzinfo=<UTC>)}, 'vertipad.vertipad_id': '고양킨텍스_1'}, {'time': {'time': neo4j.time.Time(15, 40, 0, 0, tzinfo=<UTC>)}, 'vertipad.vertipad_id': '김포공항_1'}, {'time': {'time': neo4j.time.Time(16, 5, 0, 0, tzinfo=<UTC>)}, 'vert

'The UAM1 is scheduled to be on the 김포공항_1 vertipad at 1:10pm, 2:00pm, 2:50pm, and 3:40pm. It will also be on the 고양킨텍스_1 vertipad at 1:35pm, 2:25pm, 3:15pm, and 4:05pm.'

각 몇시에 어느 vertipad에 도착 예정인지 도출됨

#### 2) 지연된 시간의 vertiport 및 vertipad 상태 (다른 uam이 있는지 없는지) 확인

In [67]:
# 5분 지연된 시간의 vertiport 상태 확인 (UAM 없음 - 1의 상황)
chain.run("""
How many UAMs are on vertiport that includes vertipad '김포공항_1' at 1:15pm?
""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (uam:UAM)-[:ON_VERTIPAD]->(vertipad:Vertipad)-[:IN_VERTIPORT]->(vertiport:Vertiport)
WHERE vertipad.vertipad_id = '김포공항_1' AND time(uam.time) = time('13:15')
RETURN COUNT(uam) AS numberOfUAMs
Full Context:
[{'numberOfUAMs': 0}]

> Finished chain.


"There are no UAMs on the vertiport that includes the vertipad '김포공항_1' at 1:15pm."

다른 UAM이 없으므로 지연된 스케줄 수용 가능

In [8]:
# 10분 지연된 시간의 vertiport 상태 확인 (uam 있음 - 2의 상황)
chain.run("""
How many UAMs are on vertiport '김포공항' at 1:20pm? And which vertipads have UAMs?
""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (uam:UAM)-[:ON_VERTIPAD]->(vertipad:Vertipad)-[:IN_VERTIPORT]->(vertiport:Vertiport {vertiport_id: '김포공항'})
WHERE uam.time = time({hour: 13, minute: 20})
RETURN count(uam) AS uam_count, collect(DISTINCT vertipad.vertipad_id) AS vertipads_with_uams
Full Context:
[{'uam_count': 4, 'vertipads_with_uams': ['김포공항_1', '김포공항_2']}]

> Finished chain.


"At 1:20pm, there are 4 UAMs on the vertiport '김포공항'. The vertipads '김포공항_1' and '김포공항_2' have UAMs."

2개의 UAM이 있다. 그러나 김포공항 vertiport에는 총 4개의 vertipad가 존재하므로 여전히 지연된 스케줄 수용 가능

In [20]:
chain.run("""
The vertiport that includes vertipad '김포공항_1' have 4 vertipads thay UAMs can park. How many UAMs can park on that vertiport at 1:20pm?
""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (vp:Vertiport)-[:IN_AREA]->(a:Area)
WHERE vp.vertipad_id = '김포공항_1'
AND a.time = time('13:20')
MATCH (vp)-[:ON_TIME]->(t:Time)
RETURN COUNT(DISTINCT vp.uam_id) AS num_uams_park
Full Context:
[{'num_uams_park': 0}]

> Finished chain.


"At 1:20pm, there are no UAMs that can park on the vertiport that includes vertipad '김포공항_1'."

### 2-2. 예정된 항로를 이탈해야 하는 경우
    1. waypoint 5-1을 예정된 time 09:13에 통과하고 있기 때문에 정상 운행으로 판단
    2. 해당 스케줄에서 도착지인 여의도 vertiport에서의 time, vertipad를 확인
    3. time1 시점에 여의도 vertiport와 vertipad1의 상태를 확인해 여의도 vertiport를 포함한 area가 unavailable임을 확인
    4. result = 예상도착시간 : time1, 착륙가능여부 : false, 착륙불가사유
    5. (여기부터 option) 주변의 다른 available하면서 vertipad의 자리가 있는 vertiport를 탐색 → vertiport2, vertipad2
    6. 있다면 스케줄에 따른 이동 시간을 고려해 vertiport2까지의 예상 도착 시간 계산 → time2

#### 1) 버티포트 '여의도'가 위치한 영역에 비행이 불가능한 시간들 조회

In [7]:
chain.run("""
When is the area that the vertiport '여의도' located unavailable?
""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (v:Vertiport {vertiport_id: '여의도'})-[:IN_AREA]->(a:Area)
MATCH (a)-[:ON_TIME]->(t:Time)
WHERE NOT a.availability
RETURN t.time
Full Context:
[{'t.time': neo4j.time.Time(12, 55, 0, 0, tzinfo=<UTC>)}, {'t.time': neo4j.time.Time(13, 10, 0, 0, tzinfo=<UTC>)}, {'t.time': neo4j.time.Time(15, 25, 0, 0, tzinfo=<UTC>)}, {'t.time': neo4j.time.Time(17, 20, 0, 0, tzinfo=<UTC>)}, {'t.time': neo4j.time.Time(18, 20, 0, 0, tzinfo=<UTC>)}]

> Finished chain.


"The area where the vertiport '여의도' is located is unavailable at the following times: 12:55 PM, 1:10 PM, 3:25 PM, 5:20 PM, and 6:20 PM."

오후 1시 10분에 여의도 버티포트 지역에 비행이 불가능한 것을 확인

#### 2) 불가능한 시간대에 도착할 예정인 UAM 조회

In [10]:
chain.run("""
What UAM is on vertiport '여의도' at 1:10pm?
""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (uam:UAM)-[:ON_VERTIPAD]->(vertipad:Vertipad)-[:IN_VERTIPORT]->(vertiport:Vertiport)
WHERE vertiport.vertiport_id = '여의도' AND uam.time = time('13:10')
RETURN uam.uam_id
Full Context:
[{'uam.uam_id': 'UAM5'}]

> Finished chain.


"The UAM on vertiport '여의도' at 1:10pm is UAM5."

#### 3) 1시 10분 이후 다시 비행이 가능한 시간대 조회

In [13]:
chain.run("""
When is the area that the vertiport '여의도' located available after 1:10pm?
""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (v:Vertiport {vertiport_id: '여의도'})-[:IN_AREA]->(a:Area)-[:ON_TIME]->(t:Time)
WHERE t.time > time('13:10')
RETURN a.area_id, t.time
Full Context:
[{'a.area_id': '서울시 영등포구', 't.time': neo4j.time.Time(13, 20, 0, 0, tzinfo=<UTC>)}, {'a.area_id': '서울시 영등포구', 't.time': neo4j.time.Time(13, 35, 0, 0, tzinfo=<UTC>)}, {'a.area_id': '서울시 영등포구', 't.time': neo4j.time.Time(13, 45, 0, 0, tzinfo=<UTC>)}, {'a.area_id': '서울시 영등포구', 't.time': neo4j.time.Time(14, 0, 0, 0, tzinfo=<UTC>)}, {'a.area_id': '서울시 영등포구', 't.time': neo4j.time.Time(14, 10, 0, 0, tzinfo=<UTC>)}, {'a.area_id': '서울시 영등포구', 't.time': neo4j.time.Time(14, 25, 0, 0, tzinfo=<UTC>)}, {'a.area_id': '서울시 영등포구', 't.time': neo4j.time.Time(14, 35, 0, 0, tzinfo=<UTC>)}, {'a.area_id': '서울시 영등포구', 't.time': neo4j.time.Time(14, 50, 0, 0, tzinfo=<UTC>)}, {'a.area_id': '서울시 영등포구', 't.time': neo4j.time.Time(15, 0, 0, 0, tzinfo=<UTC>)}, {'a.area_id': '서울시 영등포구', 't.time': neo4j.time.T

"The area where the vertiport '여의도' is located is available at the following times after 1:10pm: 1:20pm, 1:35pm, 1:45pm, 2:00pm, 2:10pm, 2:25pm, 2:35pm, 2:50pm, 3:00pm, and 3:15pm."